<a href="https://colab.research.google.com/github/villifCoder559/VGGFaceFeatures_PCA/blob/main/VGGFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu --no-cache

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install tensorflow keras_applications
!pip install layer_utils

In [ ]:
!pip install keras==2.12

In [4]:
import numpy as np
import pandas as pd
import cv2
import keras
from keras_vggface.vggface import VGGFace
from sklearn.datasets import fetch_lfw_people
from skimage.transform import resize
import faiss
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd

In [5]:
from sklearn.model_selection import train_test_split
faces = fetch_lfw_people(min_faces_per_person=70,resize=2.3,color=True)
X_train, X_test, y_train, y_test = train_test_split(faces.images, faces.target, test_size=0.1, random_state=2)

In [6]:
image_samples_train=np.copy(X_train)
image_samples_train=np.array([cv2.resize(img,(224,224)) for img in image_samples_train])
image_samples_train[:,:,:,0]=(image_samples_train[:,:,:,0]-np.mean(X_train[:,:,:,0]))/(np.std(X_train[:,:,:,0]))
image_samples_train[:,:,:,1]=(image_samples_train[:,:,:,1]-np.mean(X_train[:,:,:,1]))/(np.std(X_train[:,:,:,1]))
image_samples_train[:,:,:,2]=(image_samples_train[:,:,:,2]-np.mean(X_train[:,:,:,2]))/(np.std(X_train[:,:,:,2]))

In [7]:
image_samples_test=np.copy(X_test)
image_samples_test=np.array([cv2.resize(img,(224,224)) for img in image_samples_test])
image_samples_test[:,:,:,0]=(image_samples_test[:,:,:,0]-np.mean(X_train[:,:,:,0]))/(np.std(X_train[:,:,:,0]))
image_samples_test[:,:,:,1]=(image_samples_test[:,:,:,1]-np.mean(X_train[:,:,:,1]))/(np.std(X_train[:,:,:,1]))
image_samples_test[:,:,:,2]=(image_samples_test[:,:,:,2]-np.mean(X_train[:,:,:,2]))/(np.std(X_train[:,:,:,2]))

Possible comparisons pre-processing image after scaling 224x224:
*   Centering and Standardization
*   PCA (no,500,100,20)
*   FlatL2 or FlatIP (faiss)
*   Precision top 5,20,50,all

In [8]:
# vggface = VGGFace(model='vgg16',include_top=False,input_shape=(image_samples_train.shape[1],image_samples_train.shape[2],image_samples_train.shape[3]))
# # resized_images=[resize(img,output_shape=(224,224)) for img in faces.images]
# result_train=vggface.predict(image_samples_train)
# result_test=vggface.predict(image_samples_test)
# original_result_train=np.copy(result_train)
# original_result_test=np.copy(result_test)

In [9]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/'
# np.save(os.path.join(drive_path, 'faces.npy'), faces)
# np.save(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_train.npy'), original_result_train)
# np.save(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_test.npy'), original_result_test)

Mounted at /content/drive


In [10]:
import os
drive_path = '/content/drive/MyDrive/'
result_train=np.load(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_train.npy'),allow_pickle=True)
result_test=np.load(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_test.npy'),allow_pickle=True)

In [135]:
fixed_components=50
def create_test_reduction(n_components,result_train):
  features_reducted=result_train.reshape(result_train.shape[0],-1)
  if n_components > 0:
    tmp_PCA=PCA(n_components=n_components)
    features_reducted=np.ascontiguousarray(tmp_PCA.fit_transform(features_reducted))
  else:
    tmp_PCA=PCA(n_components=fixed_components).fit(features_reducted)
    features_reducted=custom_reduction(tmp_PCA,result_train,n_components)
  faiss.normalize_L2(features_reducted)
  return tmp_PCA,features_reducted,faiss.IndexFlatIP(features_reducted.shape[1])

In [12]:
def custom_reduction(pca,result_train,start_index):
  transformed_results = []
  for element in result_train:
    test = element.reshape(1, -1)
    # Apply PCA transformation
    transformed_result = np.dot(test - pca.mean_, pca.components_[(-start_index):].T)
    transformed_results.append(transformed_result)
  return np.array(transformed_results).reshape(result_train.shape[0],pca.components_.shape[0]+start_index)

In [171]:
pca_objs=[]
list_n_components=[0] #0 -> No PCA, -K -> Without first K eigenvectors, +K -> Use K components
features_objs=[]
index_faiss_objs=[]
for n_components in list_n_components:
  if n_components>0:
    pca,features,index=create_test_reduction(n_components,result_train)
    pca_objs.append(pca)
    features_objs.append(features)
    index_faiss_objs.append(index)
  else:
    if(n_components<0):
      pca,features,index=create_test_reduction(n_components,result_train)
      pca_objs.append(pca)
      features_objs.append(features)
      index_faiss_objs.append(index)
    else: # dummy PCA
      pca_objs.append(PCA())
      features_objs.append(result_train.reshape(result_train.shape[0],-1))
      index_faiss_objs.append(faiss.IndexFlatIP(result_train.reshape(result_train.shape[0],-1).shape[1]))
#add vectors to index
count=0
for index in index_faiss_objs:
  print(features_objs[count].shape)
  index.add(features_objs[count])
  count=count+1

(1159, 25088)


In [144]:
def compute_precision_recall(ground_truth,retrived_indices_faiss,target,retrive_all=False):
  retrived_targets_id=ground_truth[retrived_indices_faiss]
  TP=np.count_nonzero(retrived_targets_id==target)
  total_samples=np.sum(ground_truth==target)
  FP=retrived_indices_faiss.shape[1]-TP
  precision=0
  if TP+FP!=0:
    precision=TP/(TP+FP)
  recall=TP/(TP+(total_samples-TP))
  return precision,recall

In [233]:
tmp1=np.array([1,3,2,4,2,3,8])
tmp1==3


array([False,  True, False, False, False,  True, False])

In [338]:
def compute_precision_recall(ground_truth,target,index_faiss,top_k,reduced_vector,retrive_all=False):
  TP=-1
  # top_k-=1
  reduced_vector=reduced_vector.reshape(1, -1)
  # while TP==-1 or retrive_all:
  _,retrived_elements=index.search(reduced_vector, X_train.shape[0])
  # retrived_targets_id=ground_truth[retrived_elements]
  retrived_targets_id=ground_truth[retrived_elements]==target
  indices = np.where(retrived_targets_id.reshape(-1))[0]
  if(top_k>len(indices)):
    print('Face nr.',target,'max elements:',len(indices)-1,'(NOT',str(top_k)+')')
    top_k=len(indices)
  stop=indices[top_k-1]+1
  TP=top_k
  # TP=np.count_nonzero(retrived_targets_id[:stop]==target)
  total_samples=np.sum(ground_truth==target)
  # FP=retrived_elements.shape[1]-TP
  FP=stop-TP
  FN=total_samples-TP
  precision=0
  if TP+FP!=0:
    precision=TP/(TP+FP)
  recall=TP/((TP+FN))
  if retrive_all:
    top_k=top_k+1000
  if recall==1:
    retrive_all=False
  return precision,recall

In [347]:
import time
list_top_elements=[50]
i=0
j=0
k=0
precision_results=[]
recall_results=[]
name_results=[]
list_precision_results=[]
list_recall_results=[]
test_vectors=result_test.reshape(result_test.shape[0],-1)
# vector=result_test[count]
for top_k in list_top_elements:
  for index in index_faiss_objs:
    for vector in test_vectors:
      start=time.time()
      if list_n_components[i] > 0:
        reduced_vector=(vector.reshape(1,-1)-pca_objs[i].mean_)@pca_objs[i].components_.T
      else:
        if list_n_components[i] < 0:
          reduced_vector=(vector.reshape(1,-1)-pca_objs[i].mean_)@pca_objs[i].components_[-list_n_components[i]:].T
        else:
          reduced_vector=vector
      # _,retrived_elements=index.search((reduced_vector.reshape(1, -1)), top_k)
      # precision,recall=compute_precision_recall(y_train,retrived_elements,y_test[j])
      precision,recall=compute_precision_recall(y_train,y_test[j],index,top_k,reduced_vector,True)
      name_results.append(y_test[j])
      precision_results.append(precision)
      recall_results.append(recall)
      j=j+1
      tmp=time.time()-start
      # print('Time ',i,tmp)
    i=i+1
    j=0
recall_results=np.array(recall_results).reshape(len(list_top_elements),len(index_faiss_objs),test_vectors.shape[0])
precision_results=np.array(precision_results).reshape(len(list_top_elements),len(index_faiss_objs),test_vectors.shape[0])
name_results=np.array(name_results).reshape(len(list_top_elements),len(index_faiss_objs),test_vectors.shape[0])

In [344]:
def create_DataFrame(id_labels,selected_name_result,selected_precision_result,normalization_factors,index_faiss_n_components,list_n_components,index_selected_top_k_retrival):
  str_top_k=str(list_top_elements[index_selected_top_k_retrival])
  df = pd.DataFrame({
    "Faces":id_labels,
    ("P@"+str_top_k+'_'+str(list_n_components[index_faiss_n_components])):np.bincount(selected_name_result,weights=selected_precision_result)/normalization_factors,
    ("R@"+str_top_k+'_'+str(list_n_components[index_faiss_n_components])):np.bincount(selected_name_result,weights=selected_recall_result)/normalization_factors,
  })
  return df
  # if(list_n_components[index_faiss_n_components]>0):
  #   return df.style.set_caption("nr comp."+str(list_n_components[index_faiss_n_components]))
  # elif list_n_components[index_faiss_n_components]==0:
  #   return df.style.set_caption("nr comp. Full dimension")
  # else:
  #   return df.style.set_caption("W/o first "+str(-list_n_components[index_faiss_n_components])+' components')

In [345]:
# index_selected_top_k_retrival = 0 # precision_results.shape[0]
# index_faiss_n_components = 4 # precision_results.shape[1]
df_list=[]
for i in range(precision_results.shape[0]):
  for j in range(precision_results.shape[1]):
    selected_name_result = name_results[i][j]
    selected_recall_result = recall_results[i][j]
    selected_precision_result = precision_results[i][j]
    id_labels,normalization_factors = np.unique(y_test,return_counts=True)
    df=create_DataFrame(id_labels,selected_name_result,selected_precision_result,normalization_factors,j,list_n_components,i)
    df_list.append(df)

In [346]:
merged_df=df_list[0]
for i in range(1,len(df_list)):
  merged_df=pd.merge(merged_df,df_list[i])
merged_df.drop(columns=['Faces']).style.highlight_max(color = 'green', axis = 1).highlight_min(color = 'red', axis = 1)

,P@5_0,R@5_0
0,0.912698,0.073529
1,1.000000,0.025126
2,0.849346,0.044643
3,0.996032,0.010246
4,0.612458,0.050505
5,0.944444,0.080645
6,0.800214,0.038168
